In [ ]:
from esper.table_tennis.utils import *
from esper.table_tennis.motion_control import *
from esper.table_tennis.pose_utils import * 

import cv2
import random
import pickle
import pycocotools.mask as mask_util
from scipy import ndimage
import numpy as np
from scipy.signal import savgol_filter

In [ ]:
sc = Client()
video_id = 65
video = Video.objects.filter(id=video_id)[0]
video_ids = [video_id]

# load data

In [ ]:
match_intervals_all = pickle.load(open('/app/data/pkl/match_scene_intervals_dict.pkl', 'rb'))
match_intervals_A = match_intervals_all['HW_foreground']
match_intervals_B = match_intervals_all['JZ_foreground']
match_intervals_A.sort()
match_intervals_B.sort()

In [ ]:
match_ism_A = list_to_IntervalSetMapping(match_intervals_A)
match_ism_B = list_to_IntervalSetMapping(match_intervals_B)

In [ ]:
count_duration(match_ism_A) + count_duration(match_ism_B)

In [ ]:
# collect all openpose for foreground players
fid2openpose_A = {}
fid2openpose_B = {}
for interval in match_intervals_A:
    fid2pose_fg, fid2pose_bg = group_pose_from_interval(interval)
    fid2openpose_A = {**fid2openpose_A, **fid2pose_fg}
for interval in match_intervals_B:
    fid2pose_fg, fid2pose_bg = group_pose_from_interval(interval)
    fid2openpose_B = {**fid2openpose_B, **fid2pose_fg}

In [ ]:
match_scene_cls = pickle.load(open('/app/data/pkl/match_scene_cls.pkl', 'rb'))
densepose_result = pickle.load(open('/app/data/pkl/densepose_result.pkl', 'rb'), encoding='latin1')
maskrcnn_result = pickle.load(open('/app/data/pkl/maskrcnn_result.pkl', 'rb'))

In [ ]:
fid2densepose = {}
for player in densepose_result:
    fid2densepose[player['fid']] = player
fid2maskrcnn = {}
for fid, bbox in enumerate(maskrcnn_result):
    fid2maskrcnn[fid] = bbox

# Find clean sport field background

In [ ]:
for fid in range(len(match_scene_cls)):
    if match_scene_cls[fid]:
        poses = Pose.objects.filter(frame__video_id=65, frame__number=fid)
        if len(poses) == 0:
            print(fid)

In [ ]:
# clean background in 65: 39050
background = load_frame(video, 39050, [])
# imshow(frame)

# Hand annotate ball hits

In [ ]:
%matplotlib notebook

In [ ]:
match_intervals_B

In [ ]:
fid -= 5

In [ ]:
# fid = 37287
frame = load_frame(video, fid, [])
print(fid)
fid += 1
imshow(frame)

In [ ]:
hit_annotation = [
    [(35583, 806, 398, 0), (35599, 1067, 587, 1),  (35621, 937, 418, 0), (35638, 863, 537, 1), (35644, 1306, 369, 0)],
    [(36506, 802, 537, 1), (36526, 745, 469, 0), (36543, 1198, 612, 1), (36551, 946, 366, 0), 
    (36563, 854, 635, 1), (36577, 1077, 274, 0), (36590, 779, 790, 1), (36611, 464, 285, 0)],
    [(37839, 1214, 385, 0), (37858, 1108, 567, 1), (37866, 1148, 415, 0), (37878, 920, 603, 1)],
    [(37304, 740, 612, 1), (37322, 1058, 447, 0), (37332, 857, 600, 1), (37339, 1110, 408, 0), 
    (37351, 877, 665, 1), (37363, 1157, 357, 0), (37372, 726, 619, 1)]
]
def convert_hit_annotation(annot):
    annotation = []
    for traj in annot:
        annotation += [[{'fid':fid, 'hit':(x, y), 'fg':fg} for fid, x, y, fg in traj]]
    return annotation
hit_annotation = convert_hit_annotation(hit_annotation)

# draw ball trajectories

In [ ]:
videowriter = cv2.VideoWriter('/app/result/visualize_trajectory.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 8, (video.width, video.height))
for hit_traj in hit_annotation:
    print(hit_traj)
    ball_traj = interpolate_trajectory_from_hit(hit_traj)
    for ball in ball_traj:
#         frame = background.copy()
        frame = load_frame(video, ball['fid'], [])
        cv2.circle(frame, ball['pt'], 8, (0, 0, 255), -1)
        videowriter.write(frame)
videowriter.release()

# Generate motion from simple left/right control

## Label clip for left/right control demo
According to the x position, segment the clip into three types of the motion: moving left, still, moving right

In [ ]:
motion_dict = {'left': [], 'right': [], 'still': []}
for interval in match_intervals_A:
    motion_dict_i = group_motion(interval, fid2openpose_A)
    motion_dict['left'] += motion_dict_i['left']
    motion_dict['right'] += motion_dict_i['right']
    motion_dict['still'] += motion_dict_i['still']
print(len(motion_dict['left']), len(motion_dict['still']), len(motion_dict['right']))

In [ ]:
# visualize labeled clips
plt.figure(figsize=(10, 10))

for motion in motion_dict['left']:
    plt.scatter(motion['start_x'], motion['end_x'], c='r', s=motion['duration']*50)
for motion in motion_dict['right']:
    plt.scatter(motion['start_x'], motion['end_x'], c='b', s=motion['duration']*50)
for motion in motion_dict['still']:
    plt.scatter(motion['start_x'], motion['end_x'], c='g', s=motion['duration']*50)
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('Foreground player start position X', fontsize=22)
plt.ylabel('Foreground player end position X', fontsize=22)
plt.show()

## Generate motion match left/right control

In [ ]:
motion_1 = find_motion(motion_dict, 0.35, 0.65, 1, 'right', (1,1,0))
motion_2 = find_motion(motion_dict, 0.65, 0.35, 1, 'left', (1,1,0))
motion_3 = find_motion(motion_dict, 0.35, 0.35, 3, 'still', (1,1,1))

print(motion_1, motion_2, motion_3)
def motion2interval(motion):
    return (video_id, motion['start_fid'], motion['end_fid'], motion['duration'])
searched_intervals = [motion2interval(motion_1), motion2interval(motion_2), motion2interval(motion_3)]
print(searched_intervals)

In [ ]:
from esper.supercut import stitch_video_temporal
stitch_video_temporal(searched_intervals, out_path='/app/result/naive_control.mp4')

# Generate motion without hit label
Use hand annotated ball trajectory, search for any interval from the motion database

In [ ]:
H, W = video.height, video.width
motion_dict_A = []
for _, sfid, efid, _ in match_intervals_A:
    motion_traj = {}
    for fid in range(sfid, efid):
        if fid in fid2openpose_A:
            pose = fid2openpose_A[fid]
            kp = pose._format_keypoints()
            motion_traj[fid] = {'Neck': (int(kp[Pose.Neck][0]*W), int(kp[Pose.Neck][1]*H)), 
                                'RWrist': (int(kp[Pose.RWrist][0]*W), int(kp[Pose.RWrist][1]*H)),
                               'LAnkle': (int(kp[Pose.LAnkle][0]*W), int(kp[Pose.LAnkle][1]*H)),
                               'RAnkle': (int(kp[Pose.RAnkle][0]*W), int(kp[Pose.RAnkle][1]*H))}
    motion_dict_A.append(motion_traj)

In [ ]:
hit_traj = hit_annotation[3]

In [ ]:
generate_motion_without_hitlabel(sc, video, fid2densepose, motion_dict_A, hit_traj, 
                                 out_path='/app/result/naive_control.avi')

# Generate motion with hit label

In [ ]:
# prepare motion database
hit_dict = pickle.load(open('/app/data/pkl/hit_dict.pkl', 'rb'))
hit_dict_A = hit_dict['HW']
hit_dict_B = hit_dict['JZ']

In [ ]:
# select motion dict and ball trajectory
motion_dict = hit_dict_A
hit_traj = hit_dict_B[7]
hit_traj

In [ ]:
generate_motion_local(sc, video, motion_dict, hit_traj, 
                               out_path='/app/result/motion_generation/triangle_label_JZ_7.avi')

In [ ]:
generate_motion_local(sc, video, motion_dict, hit_traj, 
                               out_path='/app/result/motion_generation/triangle_label_JZ_7.avi')